In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split

# Linear Regression

In [3]:
data = pd.read_csv("Advertising.csv")
data.tail()

,TV,Radio,Sales
195,38.2,3.7,7.6
196,94.2,4.9,9.7
197,177.0,9.3,12.8
198,283.6,42.0,25.5
199,232.1,8.6,13.4


### Spilt the data into 2-way (70%, 30%)

In [4]:
Y = data.Sales
X = data.drop("Sales", axis=1)

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size = 0.5)

In [6]:
m = LR().fit(xtrain, ytrain)
m

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
m.intercept_

2.981224910668315

In [8]:
m.coef_

array([0.0461081 , 0.18452946])

# Linear Programming

In [9]:
import pyomo.environ as pyo
from pyomo.environ import *

In [10]:
b0 = m.intercept_
b1, b2 = m.coef_[0], m.coef_[1]

In [11]:
b0, b1, b2

(2.981224910668315, 0.04610810197614335, 0.18452946444006524)

In [12]:
def LPModel():
    m = AbstractModel()
    
    # mutable param
    m.b0 = Param(initialize = b0, mutable = True)
    m.b1 = Param(initialize = b1, mutable = True)
    m.b2 = Param(initialize = b2, mutable = True)
    
    # decision variables
    m.x1 = Var(within = NonNegativeReals, bounds = (0.7, 296.4))
    m.x2 = Var(within = NonNegativeReals, bounds = (0, 49.6))
    m.y1 = Var(within = NonNegativeReals)
    m.y2 = Var(within = NonNegativeReals)
    
    # constraints
    m.c1 = Constraint(expr = m.x1 + m.x2 <= 200)
    m.c2 = Constraint(expr = m.x1 - 0.7*m.x2 <= 0)
    m.c3 = Constraint(expr = m.x1 - 0.2*m.x2 >= 0)
    m.c4 = Constraint(expr = m.y1 <= 8)
    m.c5 = Constraint(expr = 2*m.y2 <= 24)
    m.c6 = Constraint(expr = 3*m.y1 + 2*m.y2 <= 36)
    m.c7 = Constraint(expr = -m.x1*m.b1 - m.x2*m.b2 + m.y1 + m.y2 <= m.b0)
    
    # objective function
    m.obj = Objective(expr = 1000*(-0.5*m.x1 - 0.2*m.x2 + 3*m.y1 + 5*m.y2), 
                     sense = maximize)
    
    return m
    

In [13]:
# m = LPModel()
m = LPModel().create_instance()
solver = SolverFactory('cplex')
result = solver.solve(m)

In [14]:
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpj962xaey
  Lower bound: 46893.836155496676
  Upper bound: 46893.836155496676
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 5
  Number of nonzeros: 15
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 0.0
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 4.6893836155e+04
  Error rc: 0
  Time: 0.022004127502441406
# ----------------------------------------------------------
#   Solution Information
# -------------------------

# Validate the result 

In [15]:
x = xtrain.values
y = ytrain.values
xt = xtest.values
yt = ytest.values

# restore the original result
x1hat = m.x1()
x2hat = m.x2()
obj = m.obj()

In [16]:
x1hat, x2hat

(9.920000000000002, 49.6)

# Calculate essential variables
#### 1. For each data point ($x_{1i}$, $x_{2i}$,$w_i$) in the validation set:

In [17]:
epv = [yt[i] - b0 - b1*xt[i][0] - b2*xt[i][1] for i in range(len(xtest))]

#### 2. For each i in the validation set:

In [18]:
wi = [b0 + b1*x1hat + b2*x2hat + epv[i] for i in range(len(xtest))]

#### 3. For each data point ($x_{1i}$, $x_{2i}$,$w_i$) in the training set

In [19]:
eti = [y[i] - b0 - b1*x[i][0] - b2*x[i][1] for i in range(len(xtrain))]

### $X^2$ test and QQ plot

### 95% CI from validation set

In [20]:
import scipy.stats

In [21]:
profits = []

for i in range(len(xtest)):   
    # fix variables
    m.x1.fix(x1hat)
    m.x2.fix(x2hat)
             
    # update parameters
    m.b1 = 0
    m.b2 = 0
    m.b0 = wi[i]
    result = SolverFactory('cplex').solve(m)
    profits.append(m.obj())

In [22]:
def GetCI(arr, confident = 0.95):
    mean = np.mean(arr)
    se = scipy.stats.sem(1.0*np.array(arr))
    n = len(arr)
    h = se * scipy.stats.t.ppf((1 + confident)/2., n-1)
    return mean ,h , mean - h, mean + h

In [23]:
mean,h ,LB, UB = GetCI(profits)
LB, obj, UB

(44480.37290982066, 46893.836155496676, 47120.74968876925)

### Verify the validation result by creating indepandent models

In [24]:
def LPModel_validation(w):
    m = ConcreteModel()
    
    # decision variables
    m.y1 = Var(within = NonNegativeReals)
    m.y2 = Var(within = NonNegativeReals)
    
    # constraints
    m.c4 = Constraint(expr = m.y1 <= 8)
    m.c5 = Constraint(expr = 2*m.y2 <= 24)
    m.c6 = Constraint(expr = 3*m.y1 + 2*m.y2 <= 36)
    m.c7 = Constraint(expr = m.y1 + m.y2 <= w)
    
    # objective function
    m.obj = Objective(expr = 1000*(3*m.y1 + 5*m.y2), 
                     sense = maximize)
    
    return m

In [25]:
profits = []
for i in range(len(xtest)):   
    m = LPModel_validation(wi[i])
    result = SolverFactory('cplex').solve(m)
    profits.append(m.obj())

In [26]:
mean ,h,_,_ = GetCI(profits)
UB = 1000*(-0.5*x1hat - 0.2*x2hat)+ mean + h 
LB = 1000*(-0.5*x1hat - 0.2*x2hat)+ mean - h 
LB, obj, UB

(44480.37290982066, 46893.836155496676, 47120.74968876925)